In [141]:
from influxdb import DataFrameClient
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.externals import joblib

host = '192.168.4.33'
port = 8086
user = ''
password = ''
db_name = 'Labview'

query_body = """

SELECT  npt, ngp,t5_average_temperature,engine_pcd ,t1_temperature, turb_air_inlet_filter_dp, gas_fuel_flow_x

FROM Unit1

WHERE time > \'2018-05-24T03:30:00Z\'

"""


LIN_REG = 'Linear Regression'
ALGORYTHM = 'LIN_REG'
# ALGORYTHM = ''
PLOT_TITLE = 'Prediction of Gas Flow. Prediction and Test'


class GasFlow_Predictor():

    def __init__(self, train_p=0.8):
        
        self.regr = LinearRegression(normalize=True)
        self.variance = -1
        self.mse = -1
        self.train_p = train_p

    def get_data_from_influx(self, host, port, user, password, db_name):

        self.client = DataFrameClient(host, port, user, password, db_name)

        self.data = self.client.query(query_body)

    def read_data(self):
        
        self.new_data = dict(self.data)

        for i in self.new_data:
            
            self.my_data = self.new_data[i]
            
    def prepare_data(self):

        self.my_data = self.my_data.reset_index()
        
        print(self.my_data.shape)

        self.my_data = self.my_data.dropna()
        
        print(self.my_data.shape)

        
        print(self.my_data.corr()[['gas_fuel_flow_x']].sort_values('gas_fuel_flow_x'))  

        self.my_data = self.my_data.set_index('index')
        
        #print(self.my_data.head())

        self.x_data = self.my_data.drop('gas_fuel_flow_x', axis = 1)
        
        print(self.x_data.columns)

        self.y_data = pd.DataFrame(self.my_data['gas_fuel_flow_x'], index = self.my_data.index)
        
        print(self.y_data.columns)

        index = int(len(self.x_data) * self.train_p)

        self.x_train = self.x_data[:index]
        
      #  print(x_train.shape)

        self.y_train = self.y_data[:index]

        self.x_test = self.x_data[index:]

        self.y_test = self.y_data[index:]
        
        return self.my_data

    


In [142]:
kettik = GasFlow_Predictor()

In [143]:
kettik.get_data_from_influx(host, port, user, password, db_name)

In [144]:
kettik.read_data()

In [145]:
aa=kettik.prepare_data()

(22949, 8)
(2487, 8)
                          gas_fuel_flow_x
engine_pcd                      -0.040733
t5_average_temperature           0.169003
ngp                              0.605326
npt                              0.635569
gas_fuel_flow_x                  1.000000
t1_temperature                        NaN
turb_air_inlet_filter_dp              NaN
Index(['engine_pcd', 'ngp', 'npt', 't1_temperature', 't5_average_temperature',
       'turb_air_inlet_filter_dp'],
      dtype='object')
Index(['gas_fuel_flow_x'], dtype='object')


In [146]:
aa.index


DatetimeIndex(['2018-05-30 06:25:16+00:00', '2018-05-30 06:25:46+00:00',
               '2018-05-30 06:26:16+00:00', '2018-05-30 06:26:46+00:00',
               '2018-05-30 06:27:16+00:00', '2018-05-30 06:27:46+00:00',
               '2018-05-30 06:28:16+00:00', '2018-05-30 06:28:46+00:00',
               '2018-05-30 06:29:16+00:00', '2018-05-30 06:29:46+00:00',
               ...
               '2018-05-31 06:43:16+00:00', '2018-05-31 06:43:46+00:00',
               '2018-05-31 06:44:16+00:00', '2018-05-31 06:44:46+00:00',
               '2018-05-31 06:45:16+00:00', '2018-05-31 06:45:46+00:00',
               '2018-05-31 06:46:16+00:00', '2018-05-31 06:46:46+00:00',
               '2018-05-31 06:47:16+00:00', '2018-05-31 06:47:46+00:00'],
              dtype='datetime64[ns, UTC]', name='index', length=2487, freq=None)

In [147]:
b = dict()
c = dict()

In [148]:
b['sss'] = aa[0:1].index

In [149]:
b['sss']

DatetimeIndex(['2018-05-30 06:25:16+00:00'], dtype='datetime64[ns, UTC]', name='index', freq=None)

In [150]:
c['www'] = aa[2486:2487].index

In [151]:
c['www']

DatetimeIndex(['2018-05-31 06:47:46+00:00'], dtype='datetime64[ns, UTC]', name='index', freq=None)

In [152]:
c

{'www': DatetimeIndex(['2018-05-31 06:47:46+00:00'], dtype='datetime64[ns, UTC]', name='index', freq=None)}

In [153]:
a=c['www']-b['sss']

In [154]:
b['lll'] = aa[23:24].index

In [155]:
type(a)

pandas.core.indexes.timedeltas.TimedeltaIndex

In [156]:
kunder = a.days

In [157]:
hours=(a.seconds / 3600)


In [158]:
kunder

Int64Index([1], dtype='int64', name='index')

In [159]:
hours

Float64Index([0.375], dtype='float64', name='index')

In [160]:
total = kunder * 24 + hours 

In [161]:
total.values

array([24.375])

In [162]:
pd_a = pd.DataFrame({'time' : a}, index = aa[0:1].index)

In [163]:
pd_a

,time
index,
2018-05-30 06:25:16+00:00,1 days 00:22:30


In [164]:
pd_total = pd.DataFrame({'radio' : total.values}, index = aa[0:1].index)

In [165]:
pd_total

,radio
index,
2018-05-30 06:25:16+00:00,24.375


In [166]:
pd_total.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2018-05-30 06:25:16+00:00 to 2018-05-30 06:25:16+00:00
Data columns (total 1 columns):
radio    1 non-null float64
dtypes: float64(1)
memory usage: 16.0 bytes


In [167]:
zhost = '192.168.4.33'

zport = 8086

zuser = ''

zpassword = ''

zdb_name = 'Online_Classification'

In [168]:
from influxdb import DataFrameClient


In [169]:
writer_client = DataFrameClient(zhost, zport, zuser, zpassword, zdb_name)


In [176]:
up = writer_client.write_points(pd_total, 'www')

In [171]:
def days_hours_minutes(td):
    days = td.days * 24
    hours = td.seconds / 3600
    total = days + hours
    return total.values

In [172]:
rus = days_hours_minutes(a)

In [173]:
rus

array([24.375])

In [174]:
pd_total = pd.DataFrame({'zzz' : days_hours_minutes(a)}, index = aa[0:1].index)

In [175]:
pd_total

,zzz
index,
2018-05-30 06:25:16+00:00,24.375
